# Berechnung einer absoluten Positionierung mit Code-Messungen, Teil 2

In [265]:
%%capture
# Requirements 
# pip install git+https://github.com/GNSSpy-Project/gnsspy
# pip install pyunpack
# pip install georinex

# Libs
import gnsspy as gp
import numpy as np
import georinex
import geopandas
import matplotlib.pyplot as plt
import math
import pandas as pd

# Params
np.set_printoptions(formatter={'float': '{: 0.5f}'.format})
plt.rcParams['figure.dpi'] = 300

In [266]:
%%capture
# Dataimport
station = gp.read_obsFile("./data/ONSA0320.11O")
ephemerides = georinex.load("./data/G3_11032.PRE")
clock = gp.read_clockFile("./data/cod16212.clk")

In [267]:
# get dfs for each epoch
clock_epoch_0 = clock[clock.Epoch == "2011-02-01 00:00:00"]
clock_epoch_1 = clock[clock.Epoch == "2011-02-01 00:15:00"]
clock_epoch_2 = clock[clock.Epoch == "2011-02-01 00:30:00"]
clock_epoch_3 = clock[clock.Epoch == "2011-02-01 00:45:00"]
clock_epoch_4 = clock[clock.Epoch == "2011-02-01 01:00:00"]

epochs_clock = list((clock_epoch_0, clock_epoch_1, clock_epoch_2, clock_epoch_3, clock_epoch_4))

for epoch in epochs_clock:
    epoch.set_index(epoch.index.str.replace("G", "").astype(int), inplace=True)
    epoch = epoch.drop("Epoch", axis=1)

ephemerides_epoch_0 = ephemerides.sel(time="2011-02-01T00:00:00.000000000")
ephemerides_epoch_1 = ephemerides.sel(time="2011-02-01T00:15:00.000000000")
ephemerides_epoch_2 = ephemerides.sel(time="2011-02-01T00:30:00.000000000")
ephemerides_epoch_3 = ephemerides.sel(time="2011-02-01T00:45:00.000000000")
ephemerides_epoch_4 = ephemerides.sel(time="2011-02-01T01:00:00.000000000")

epochs_ephemerides = list((ephemerides_epoch_0, ephemerides_epoch_1, ephemerides_epoch_2, ephemerides_epoch_3, ephemerides_epoch_4))

In [268]:
# Clean all P1 / P2 for the epochs
P1_epoch_0 = station.observation.P1["2011-02-01 00:00:00"]
P1_epoch_0 = P1_epoch_0.filter(like='G', axis=0).dropna()
P2_epoch_0 = station.observation.P2["2011-02-01 00:00:00"]
P2_epoch_0 = P2_epoch_0.filter(like='G', axis=0).dropna()

P1_epoch_1 = station.observation.P1["2011-02-01 00:15:00"]
P1_epoch_1 = P1_epoch_1.filter(like='G', axis=0).dropna()
P2_epoch_1 = station.observation.P2["2011-02-01 00:15:00"]
P2_epoch_1 = P2_epoch_1.filter(like='G', axis=0).dropna()

P1_epoch_2 = station.observation.P1["2011-02-01 00:30:00"]
P1_epoch_2 = P1_epoch_2.filter(like='G', axis=0).dropna()
P2_epoch_2 = station.observation.P2["2011-02-01 00:30:00"]
P2_epoch_2 = P2_epoch_2.filter(like='G', axis=0).dropna()

P1_epoch_3 = station.observation.P1["2011-02-01 00:45:00"]
P1_epoch_3 = P1_epoch_3.filter(like='G', axis=0).dropna()
P2_epoch_3 = station.observation.P2["2011-02-01 00:45:00"]
P2_epoch_3 = P2_epoch_3.filter(like='G', axis=0).dropna()

P1_epoch_4 = station.observation.P1["2011-02-01 01:00:00"]
P1_epoch_4 = P1_epoch_4.filter(like='G', axis=0).dropna()
P2_epoch_4 = station.observation.P2["2011-02-01 01:00:00"]
P2_epoch_4 = P2_epoch_4.filter(like='G', axis=0).dropna()

epochs_P1 = list((P1_epoch_0, P1_epoch_1, P1_epoch_2, P1_epoch_3, P1_epoch_4))
epochs_P2 = list((P2_epoch_0, P2_epoch_1, P2_epoch_2, P2_epoch_3, P2_epoch_4))

In [269]:
# Helper funcs for satellite Positions
# Consts
omega_e = 7.292115e-5 #s^-1
c = 299792458 #m/s

def calculateSatPos(earth_fixed_coords, sat_velocities):
    omega_e = 7.292115e-5 #s^-1
    c = 299792458 #m/s
    
    earth_fixed_coords_si = earth_fixed_coords * 1000 # km to m
    sat_velocities_si = sat_velocities / 10 # dm/s to m/s
    
    tau = math.dist(np.array(station.approx_position), earth_fixed_coords_si) / c
    sat_coords = np.array(earth_fixed_coords_si).T - np.array( (sat_velocities_si + (omega_e * np.array([-earth_fixed_coords_si[1], earth_fixed_coords_si[0], 0]))) * tau)

    return sat_coords

# Helper funcs for tropospherical correction
# Rotation matrices
def ry(a): return np.matrix([[np.cos(a), 0, -np.sin(a)], [0, 1, 0], [np.sin(a), 0, np.cos(a)]])
def rz(a): return np.matrix([[np.cos(a), np.sin(a), 0], [-np.sin(a), np.cos(a), 0], [0, 0, 1]])

def calculateLatLong(earth_fixed_coords):
    x,y,z = earth_fixed_coords

    lat = math.degrees(math.atan2(z, math.sqrt(x**2 + y**2)))
    lon = math.degrees(math.atan2(y, x))

    return lat, lon

# Topo Coords
def calculateTopoCoords(stat_coord_earth_fixed, earth_fixed_coords_at_send):
    lat_s, lon_s = calculateLatLong(stat_coord_earth_fixed)
    lat_s, lon_s = math.radians(lat_s), math.radians(lon_s)

    # Calculate N, E, U
    topo_coords = ry((math.pi / 2) - lat_s) @ rz(lon_s) @ (earth_fixed_coords_at_send - stat_coord_earth_fixed).T
    n = -topo_coords[0,0]
    e = topo_coords[0,1]
    u = topo_coords[0,2]
    
    return (n, e, u)

# Zenitwinkel
def calculateZn(topo_coords):
    n, e, u = topo_coords
    return math.degrees(math.atan2(math.sqrt(n**2 + e**2), u))

# Tropo delay
def calculateTropDelay(angle):
    return 2.4 / math.cos(math.radians(angle))

# Relativistic delay
def calculateRelativistics(coord, velocity):
    velocity = velocity / 10 # dm/s to m/s
    return 2 * (coord @ velocity.T) / c

In [270]:
# Calculate Sat positions for each epoch for each satellite in Interval A
sat_coords_at_send = list()
velo_at_send = list()
for epoch in epochs_ephemerides: # for each epoch
    epoch_helper = list()
    velo_helper = list()
    for j, coord in enumerate(epoch.position): # for each coordinate
        #print("koord", coord, "velo", epoch.velocity[j])
        epoch_helper.append(calculateSatPos(coord, epoch.velocity[j])) # calculate sat position at send time
        velo_helper.append(np.array(epoch.velocity[j]))
    sat_coords_at_send.append(np.array(epoch_helper))
    velo_at_send.append(velo_helper)

# sat_coords_at_send[epoch][satellite] example epoch 4 with g25: sat_coords_at_send[4][24])

# Calculate Tropo delay for each epoch for each satellite in Interval A
tropo_delay = list()
for epoch in sat_coords_at_send: # for each epoch
    epoch_helper = list()
    for i, coords in enumerate(epoch): # for each coordinate
        epoch_helper.append(calculateTropDelay(calculateZn(calculateTopoCoords(station.approx_position, coords))))
    tropo_delay.append(np.array(epoch_helper))

#print(tropo_delay[0][24])
#print(np.array(calculateTopoCoords(np.array(station.approx_position) * 1000, sat_coords_at_send[0][24]))) # should be 0,0,0
#print(np.array(calculateZn(calculateTopoCoords(np.array(station.approx_position), sat_coords_at_send[0][24])))) # should be 63.26054

# Calculate Relativistic delay for each epoch for each satellite in Interval A
relativistic_delay = list()
for i, epoch in enumerate(sat_coords_at_send): # for each epoch
    epoch_helper = list()
    for j, coords in enumerate(epoch): # for each velocity
        epoch_helper.append(calculateRelativistics(coords, velo_at_send[i][j]))
    relativistic_delay.append(np.array(epoch_helper))

# print(relativistic_delay[0][24])

In [271]:
def calculateP3k(p_1, p_2):
    f_0 = 10.23 * 10**6
    f_1 = 154 * f_0
    f_2 = 120 * f_0
    return (f_1**2 * p_1 - f_2**2 * p_2) / (f_1**2 - f_2**2)

def calculateI0_i(stat_coord_earth_fixed, earth_fixed_coords_at_send, clock_bias_i, epoch_i, index):
    c = 299792458 #m/s
    
    tau = math.dist(np.array(station.approx_position), earth_fixed_coords_at_send) / c
    trop = tropo_delay[epoch_i][index]
    relat = relativistic_delay[epoch_i][index]

    return tau + trop + relat - clock_bias_i * c

# Calculate a-priori clock bias for each epoch for each satellite in Interval A
p3k_i = list()
I_0_i = list()
for i, epoch in enumerate(epochs_ephemerides): # for each epoch
    epoch_helper = list()
    epoch_helper_i0 = list()
    observed_satellites = epochs_P1[i].index.str.replace('G', '').astype(int)
    for j, coords in enumerate(epoch.position): # for each delta
        if j in observed_satellites:
            # calculate P3_i
            index = str(j)
            if j < 10: index = "0" + index
            p_1, p_2 = epochs_P1[i]["G" + index], epochs_P2[i]["G" + index]
            epoch_helper.append((index, calculateP3k(p_1, p_2)))

            # Calculate I0_i
            epoch_helper_i0.append((index, calculateI0_i(station.approx_position, sat_coords_at_send[i][j], epochs_clock[i].iloc[j].DeltaTSV, i, j)))
    p3k_i.append(pd.DataFrame(epoch_helper).rename(columns={0: "Satellite", 1: "P3_k"}))
    I_0_i.append(pd.DataFrame(epoch_helper_i0).rename(columns={0: "Satellite", 1: "I_0"}))


#print(p3k_i[0])
#print(I_0_i[0])

In [302]:
# Ausgleichsrechnung
## Init I tilde
I_dash = list()
for i, p3k in enumerate(p3k_i)
    satellites = p3k.Satellite
    I_dash_i = p3k.P3_k - I_0_i[i].I_0
    concat = pd.DataFrame({"Satellite": satellites, "I_dash": I_dash_i})
    I_dash.append(concat)

## Initialize all P matrices
P_epochs = list()
zn = list()
for i, I_dash_i in enumerate(I_dash):
    zn_list = list()
    for j, satellite in enumerate(I_dash_i.I_dash):
        zn_list.append(calculateZn(calculateTopoCoords(station.approx_position, sat_coords_at_send[i][I_dash[i].Satellite.astype(int)[j] - 1]))) # ZN of satellite i
    P_epochs.append( np.diag(np.cos(np.radians((np.array(zn_list))))**2 ))
    
## Initialize A matrices
A_epochs = list()
for i, I_dash_i in enumerate(I_dash):
    A_epochs.append(np.full((len(I_dash_i)), c))

## do your thing
delta_tk_epochs = list()
for i, I_dash_i in enumerate(I_dash):
    

In [303]:
A_epochs

[array([299792458, 299792458, 299792458, 299792458, 299792458, 299792458,
        299792458, 299792458, 299792458]),
 array([299792458, 299792458, 299792458, 299792458, 299792458, 299792458,
        299792458, 299792458, 299792458]),
 array([299792458, 299792458, 299792458, 299792458, 299792458, 299792458,
        299792458]),
 array([299792458, 299792458, 299792458, 299792458, 299792458, 299792458,
        299792458, 299792458, 299792458, 299792458]),
 array([299792458, 299792458, 299792458, 299792458, 299792458, 299792458,
        299792458, 299792458, 299792458, 299792458])]

In [286]:
zn[0]

array([ 74.45153,  23.78258,  17.05099,  53.96859,  78.13982,  64.93862,
        84.48380,  63.26054,  38.88737])

In [277]:
calculateZn(calculateTopoCoords(station.approx_position, sat_coords_at_send[0][I_dash[0].Satellite.astype(int)[7] - 1]))

63.26053901129386

In [276]:
I_dash[0].Satellite.astype(int)

0     4
1     9
2    12
3    14
4    15
5    17
6    22
7    25
8    27
Name: Satellite, dtype: int64